<div style='background-image: url("header.png") ; padding: 0px ; background-size: cover ; border-radius: 5px ; height: 250px'>
    <div style="float: right ; margin: 50px ; padding: 20px ; background: rgba(255 , 255 , 255 , 0.7) ; width: 50% ; height: 150px">
        <div style="position: relative ; top: 50% ; transform: translatey(-50%)">
            <div style="font-size: xx-large ; font-weight: 900 ; color: rgba(0 , 0 , 0 , 0.8) ; line-height: 100%">Tutorial by Mondaic</div>
            <div style="font-size: large ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.5)">For Salvus version 0.11.25</div>
        </div>
    </div>
</div>

# Basic Meshing API

## The most basic mesh to run with Salvus

This exercise shows the most basic mesh with all necessary commands to put material properties and boundaries ready to run with Salvus.

In [ ]:
# initialize notebook
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams["figure.figsize"] = (10, 8)

### 2D

Set basic input parameters for a rectangular domain with homogeneous material properties.

In [ ]:
vp = 1700  # p-wave velocity in m/s
vs = 1000  # s-wave velocity in m/s
rho = 2700  # density in kg / m^3

max_x = 10000  # domain size in x direction
max_y = 5000  # domain size in y direction

fmax = 1.0  # maximum frequency in Hz
elements_per_wavelength = 2.0  # resolution criterion

Compute element size and number of elements needed according to the resolution criterion.

In [ ]:
hmax = vs / fmax / elements_per_wavelength
nelem_x = int(max_x / hmax)
nelem_y = int(max_y / hmax)

In the mesher, we build unstructured meshes by combining multiple structured grids. For the simplest case here, we build a single structured grid and plot it.

In [ ]:
from salvus.mesh.structured_grid_2D import StructuredGrid2D

sg = StructuredGrid2D.rectangle(
    nelem_x=nelem_x, nelem_y=nelem_y, max_x=max_x, max_y=max_y
)
sg.plot(show=True)

Then, we convert the structured grid to an unstructured mesh (trivial in this special case, but uses the same general routines).

In [ ]:
m = sg.get_unstructured_mesh()

Attach the material properties, constant here, but using the general array storage on element nodes.

In [ ]:
m.attach_field("VP", vp * np.ones(m.npoint))
m.attach_field("VS", vs * np.ones(m.npoint))
m.attach_field("RHO", rho * np.ones(m.npoint))

# this is necessary to tell salvus that this is elastic material everywhere. Set to 1 to make it fluid.
m.attach_field("fluid", np.zeros(m.nelem))

Find outer Boundaries of the domain assuming the domain is rectangular.

In [ ]:
m.find_side_sets(mode="cartesian")

Open a preview inline, showing all parameters stored on the mesh.

In [ ]:
m

### 3D

The interfaces are very much the same as in 2D, so less comments.

In [ ]:
vp = 1700  # p-wave velocity in m/s
vs = 1000  # s-wave velocity in m/s
rho = 2700  # density in kg / m^3

max_x = 10000  # domain size in x direction
max_y = 5000  # domain size in y direction
max_z = 3000  # domain size in y direction

fmax = 1.0  # maximum frequency in Hz
elements_per_wavelength = 2.0  # resolution criterion

In [ ]:
hmax = vs / fmax / elements_per_wavelength
nelem_x = int(max_x / hmax)
nelem_y = int(max_y / hmax)
nelem_z = int(max_z / hmax)

Build a structured Grid. Don't plot it, because in 3D matplotlib is to slow.

In [ ]:
from salvus.mesh.structured_grid_3D import StructuredGrid3D

sg = StructuredGrid3D.cube(
    nelem_x=nelem_x,
    nelem_y=nelem_y,
    nelem_z=nelem_z,
    max_x=max_x,
    max_y=max_y,
    max_z=max_z,
)

In [ ]:
m = sg.get_unstructured_mesh()

In [ ]:
m.attach_field("VP", vp * np.ones(m.npoint))
m.attach_field("VS", vs * np.ones(m.npoint))
m.attach_field("RHO", rho * np.ones(m.npoint))

In [ ]:
m.find_side_sets(mode="cartesian")
m

## Meshes composed from multiple structured grids

The general strategy in the mesher is to compose unstructured meshes from multiple structured grids. For example, a mesh with a doubling can be thought of as two structured rectilinear grids connected by a doubling grid. The doubling is also considered as a structured grid, with some elements masked out and nodes moved. Nodes that appear multiple times are reduced to single node by lexicographic sorting when converting to an unstructured mesh.

In [ ]:
from salvus.mesh.skeleton import Skeleton

Start by creating three structured grids

In [ ]:
max_x = 1000.0
y = np.array([0.0, 500.0, 750.0, 1000.0])
nelem_x = 5
nelem_y = np.array([2, 2])

sg1 = StructuredGrid2D.rectangle(
    nelem_x, nelem_y[0], min_x=0, max_x=max_x, min_y=y[0], max_y=y[1]
)

sg2 = StructuredGrid2D.cartesian_doubling_layer(
    nelem_x, min_x=0, max_x=max_x, min_y=y[1], max_y=y[2]
)

sg3 = StructuredGrid2D.rectangle(
    nelem_x * 2, nelem_y[1], min_x=0, max_x=max_x, min_y=y[2], max_y=y[3]
)


# plot using different colors
sg1.plot(edgecolor="r")
sg2.plot(edgecolor="b")
sg3.plot(edgecolor="g")

plt.show()

Collect the structured grids in a Skeleton and convert to an unstructured mesh.

In [ ]:
sk = Skeleton([sg1, sg2, sg3])
m = sk.get_unstructured_mesh()
m